# Reef community mapping 



![OTR](data/satellite_mosaic_and_bathymetry.jpg)

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import cv2
from osgeo import gdal, gdal_array
import rasterio
from matplotlib import pyplot as plt
from rasterio.plot import show

#Clustering packages
# from sklearn import cluster
# from sklearn.mixture import GaussianMixture as GMM
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.cluster import DBSCAN

import reef_insight as reef

## Loading the remote sensing data

In [ ]:
mosaic_path = 'data/mosaic.tif'
bathymetry_path ='data/bathymetry.tif'

Loading the reef-mosaic as RGB 

In [ ]:
reef_mosaic_benthic = cv2.imread(mosaic_path)
reef_mosaic_benthic = cv2.cvtColor(reef_mosaic_benthic, cv2.COLOR_BGR2RGB)

fig, ax1 = plt.subplots(1,1,figsize=(10,7))
plt.imshow(reef_mosaic_benthic)
ax1.tick_params(axis="x", labelsize=8)
ax1.tick_params(axis="y", labelsize=8)
plt.show()

Loading the bathymetry data

In [ ]:
bathymetry_raster = rasterio.open(bathymetry_path)
bathymetry_raster_visual= bathymetry_raster.read([1])

fig, ax1 = plt.subplots(1,1,figsize=(10,7))
show(bathymetry_raster_visual)
ax1.tick_params(axis="x", labelsize=8)
ax1.tick_params(axis="y", labelsize=8)
plt.show()

bathymetry_raster = bathymetry_raster_visual.reshape(bathymetry_raster_visual.shape[1],bathymetry_raster_visual.shape[2],bathymetry_raster_visual.shape[0])
bathymetry_raster = np.uint16(bathymetry_raster)

Concatenting bathymetric information with the reef mosaic for geomorphic mapping

In [ ]:
reef_mosaic_geomorphic = cv2.resize(reef_mosaic_benthic, (885,472))
reef_mosaic_geomorphic = np.concatenate((reef_mosaic_geomorphic,bathymetry_raster), axis=2)

## Benthic maps  

number_of_cluster = 4


### Kmeans Result

In [ ]:
kmeans_labels = reef.kmeans_mosaic(reef_mosaic_benthic,4)
reef.show_cluster_output(kmeans_labels,reef_mosaic_benthic)

### Gaussian mixture model clustering result

In [ ]:
gmm_labels_benthic = reef.gmm_mosaic(reef_mosaic_benthic,4,'full')
reef.show_cluster_output(gmm_labels_benthic,reef_mosaic_benthic)

### Hierarchichal agglomerative clustering result

In [ ]:
hac_labels,mosaic_hac = reef.hac_mosaic(reef_mosaic_benthic,4)
reef.show_cluster_output(hac_labels,mosaic_hac)

## Geomorphic maps  

number_of_cluster = 7

### Kmeans Result

In [ ]:
kmeans_labels = reef.kmeans_mosaic(reef_mosaic_geomorphic,7)
reef.show_cluster_output(kmeans_labels,reef_mosaic_geomorphic)

### Gaussian mixture model clustering result

In [ ]:
gmm_labels_geomorphic = reef.gmm_mosaic(reef_mosaic_geomorphic,7,'full')
reef.show_cluster_output(gmm_labels_geomorphic,reef_mosaic_geomorphic)

### Hierarchichal agglomerative clustering result

In [ ]:
hac_labels,mosaic_hac = reef.hac_mosaic(reef_mosaic_geomorphic,7)
reef.show_cluster_output(hac_labels,mosaic_hac)

## Visual comparision with existing maps 

Gaussian Mixture model example


In [ ]:
# 1. Benthic Map

label_to_color = {
    0: [202, 210, 212],  # sea - gray
    1: [255, 255, 191],  # sand - light yellow
    2: [180, 155, 58],  # rock - brown
    3: [255, 255, 191],  #

}
labels = {0:'ocean',1:'sand',2:'rock/rubble',3:'misc'}

height, width,_ = reef_mosaic_benthic.shape
gmm_labels_benthic_reshaped = gmm_labels_benthic.reshape(reef_mosaic_benthic[:, :, 0].shape)
refined_gmm_benthic_map = np.zeros((height, width, 3), dtype=np.uint8)

for gray, rgb in label_to_color.items():
    refined_gmm_benthic_map[gmm_labels_benthic_reshaped == gray, :] = rgb

# Visualising
reef.show_image(refined_gmm_benthic_map)

In [ ]:
#2. Geomorphic Map
label_to_color = {
    
    0: [146,115,157],  #  purple [146, 115, 157] - reef flat
    1: [202,210,212],  #  ocean [202, 210, 212] - ocean
    2: [20,186,164],  
    3: [202,210,212],  #  green [20, 186, 164]   - reef slope 
    4: [202,210,212],
    5: [171,219,227],  #  light blue [171,219,227] - plateau/lagoon    
    6: [202,210,212],  

}

height, width,_ = reef_mosaic_geomorphic.shape
gmm_labels_geomorphic_reshaped = gmm_labels_geomorphic.reshape(reef_mosaic_geomorphic[:, :, 0].shape)
refined_gmm_geomorphic_map = np.zeros((height, width, 3), dtype=np.uint8)

for gray, rgb in label_to_color.items():
    refined_gmm_geomorphic_map[gmm_labels_geomorphic_reshaped == gray, :] = rgb

# Visualising
reef.show_image(refined_gmm_geomorphic_map)

In [ ]:
import imageio as iio

allen_coral_atlas_benthic_map = iio.v2.imread('data/benthic_map_legend.JPG')
allen_coral_atlas_geomorphic_map = iio.v2.imread('data/geomorphic_map_legend.JPG')

f, axarr = plt.subplots(nrows=2, ncols=2,figsize=(12,12))

axarr[0,0].imshow(allen_coral_atlas_benthic_map)
axarr[0,1].imshow(refined_gmm_benthic_map)
axarr[1,0].imshow(allen_coral_atlas_geomorphic_map)
axarr[1,1].imshow(refined_gmm_geomorphic_map)